In [ ]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from typing import List, T
from sklearn.model_selection import train_test_split

In [ ]:
dataset_path = 'caltech-101/101_ObjectCategories'
test_size = 0.2
k = 500

In [ ]:
categories = []
for root, dirs, files in os.walk(dataset_path):
    for dir in dirs:
        
        for cat_root, cat_dirs, cat_files in os.walk(os.path.join(dataset_path, dir)):
            
            if len(cat_files) >= 100:
                categories.append(dir)
                print(dir, len(cat_files))
              

      

In [ ]:
def get_file_paths(dataset_path : str, category : str, test_size : float) -> List[str]:
    """ 
    returns names of files for test and train datasets of particular category
    """
    files_paths = []
    for root, dirs, files in os.walk(os.path.join(dataset_path, category)):
        for name in files:
            files_paths.append(os.path.join(root, name))
    train_files, test_files = train_test_split(files_paths, test_size=test_size)
    return train_files, test_files

def load_images(files_paths : List[str]) -> List[np.ndarray]:
    images = []
    for file in files_paths:
        img = cv2.imread(file,0)
        images.append(img)
    return images
        

In [ ]:
# creating train and test datasets 
train_images = {}
test_images = {}
for cat in categories:
    train_files, test_files = get_file_paths(dataset_path, cat, test_size)
    train_images[cat] = load_images(train_files)
    test_images[cat] = load_images(test_files)

In [ ]:
def sift_features(images):
    descriptor_dict = {}
    descriptor_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for category, cat_images in images.items():
        features = []
        for img in cat_images:
            kp, des = sift.detectAndCompute(img,None)
            if (des is not None):
                descriptor_list.extend(des)
                features.append(des)
            else: 
                print(f'image number {cat_images.index(img)} has no descriptors')
        descriptor_dict[category] = features
    return [descriptor_list, descriptor_dict]

print('train')
descriptor_list, all_bovw_feature = sift_features(train_images) 
print('test')
_, test_bovw_feature = sift_features(test_images)
# descriptor_list is needed for creating clustering centers, so we only take them from train dataset
# bovw_feature for train and test are needed for classification of the image 

In [ ]:
# Returns an array that holds central points (visual words)
def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_ 
    return visual_words
  
visual_words = kmeans(k, descriptor_list) 

In [ ]:
def find_index(feature : np.ndarray, centers : np.ndarray):
    # returns an index of the visual word that is the most similar to a feature 
    minimum = 0
    start_min = True 
    min_index = 0 
    for idx, center in enumerate(centers): 
        distance_c = distance.cosine(center, feature)
        if start_min:
            minimum = distance_c
            start_min = False 
        else: 
            minimum = min(minimum, distance_c)
            if minimum == distance_c:
                min_index = idx

    return min_index

In [ ]:
# Takes 2 parameters: 
# - a dictionary that holds the descriptors that are separated class by class 
# - an array that holds the central points (visual words) of the k means clustering
# Returns a dictionary that holds the histograms for each images that are separated class by class. 
def image_class(all_bovw, centers):
    bovw = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            for feature in img:
                ind = find_index(feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        bovw[key] = category
    return bovw
    
# Creates histograms for train data    
bovw_train = image_class(all_bovw_feature, visual_words) 
# Creates histograms for test data
bovw_test = image_class(test_bovw_feature, visual_words) 

In [ ]:
import json

def save_to_json(file_name, bovw):
    bovw_list = {cat : list(bovw[cat][0]) for cat in bovw.keys()}
    json_object = json.dumps(bovw_list)
    with open(file_name, 'w') as outfile:
        outfile.write(json_object)

save_to_json('bovw_test.json', bovw_test)
save_to_json('bovw_train.json', bovw_train)
 

In [ ]:
def common_words_counter(hist1, hist2):
    cnt = 0
    for idx, word in enumerate(hist1):
        cnt += min(hist1[idx], hist2[idx])

In [ ]:
# 1-NN algorithm. We use this for predict the class of test images.
# Takes 2 parameters
# - images : feature vectors of train images 
# - tests : feature vectors of test images
# Returns an array that holds number of test images, number of correctly predicted images and records of class based images respectively
def knn(images, tests):
    num_test = 0
    correct_predict = 0
    class_based = {}
    
    for test_category, test_histograms_list in tests.items():
        class_based[test_category] = [0, 0] # [correct, all]
        for test_histogram in test_histograms_list:
            min_start = True 
            for train_category, train_histograms_list in images.items():
                for train_histogram in train_histograms_list:
                    if min_start:
                        minimum = distance.euclidean(test_histogram, train_histogram)
                        pred = train_category
                        min_start = False
                    else:
                        dist = distance.euclidean(test_histogram, train_histogram)
                        if(dist < minimum):
                            minimum = dist
                            pred = train_category
            
            if(test_category == pred):
                correct_predict += 1
                class_based[test_category][0] += 1
            num_test += 1
            class_based[test_category][1] += 1

    return [num_test, correct_predict, class_based]
    
# Call the knn function    
results_bowl = knn(bovw_train, bovw_test) 

In [ ]:
# Calculates the average accuracy and class based accuracies.  
def accuracy(results):
    avg_accuracy = (results[1] / results[0]) * 100
    print("Average accuracy: %" + str(avg_accuracy))
    print("\nClass based accuracies: \n")
    for key,value in results[2].items():
        acc = (value[0] / value[1]) * 100
        print(key + " : %" + str(acc))
        
# Calculates the accuracies and write the results to the console.       
accuracy(results_bowl) 

In [ ]:
from matplotlib import pyplot as plt
fig, axes = plt.subplots(6, 2)
fig.set_size_inches(20, 20)
for ax, cat in zip(axes, categories):
    ax[0].hist(bovw_train[cat], bins=k)
    ax[0].set_title(f'{cat} train histogram')
    ax[1].hist(bovw_test[cat], bins=k)
    ax[1].set_title(f'{cat} test histogram')

plt.show()